In [ ]:
import natsort
import numpy as np
import pandas as pd
import param
import plotly.express as px
import plotly.graph_objects as go
import re
import traceback
import panel as pn
pn.extension("plotly")
from io import BytesIO
from io import StringIO
from bokeh.models.widgets.tables import NumberFormatter
from sklearn.decomposition import PCA

In [ ]:
#e.findall('(\D+)',k)
#isteEe = ["02", "33", "3"]
#items.zfill(2) for items in listeEe]
#e.findall('(\d+)',k)[0].zfill(2)
#=[[re.match(name_pattern, col).group("frac")] for col in df_index_LFQ.columns]
#isteeeee = list(list(zip(*k))[0])
#items.zfill(3) for items in listeeeee]
#items.zfill(2) for items in [re.match(name_pattern, col).group("frac")] for col in df_index_LFQ.columns]

In [ ]:
#stringg= ["Ratio H/L count MAP1_03K", "Ratio H/L variability [%] MAP1_03K", "Ratio H/L count MAP1_03K", "Ratio H/L variability [%] MAP1_03K"]
#stringg = LFQ intensity Control_Cyt_1
#[re.findall(sets_SILAC, stringg) for col in stringg]
#k=re.match(".* (?P<rep>.*)_(?P<frac>.*)", stringg).group("rep")
#k
#[[s for s in sets_SILAC if col.startswith(s)][0] for col in stringg]
#[re.match(name_pattern_SILAC, col).group("frac")]
#[re.match("(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)", col).group("rep") for col in stringg]
#[re.match("(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)", col).group("sets") for col in stringg]


#bool([re.match(name_pattern_SILAC, col) for col in i_class.df_original.columns])
#any([[col.startswith(s) for s in sets_SILAC] for col in i_class.df_original.columns])

#[items.zfill(3) for items in ["03K", "12K"]]

#bool([re.match(name_pattern_SILAC, col) for col in i_class.df_original.columns])

#[re.match(name_pattern_SILAC, col).group("sets") for col in df_index_SILAC.columns]
#[col for col in df_index.columns if [re.match(name_pattern_SILAC, col).group("") for s in sets_SILAC]]
#len([[s for s in sets_LFQ if re.match(name_pattern_SILAC, col)][0] for col in df_index.columns])


#sets_SILAC = ["Ratio H/L variability [%]", "Ratio H/L count", "Ratio H/L"]
#"type_count_silac": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
#            "type_var_silac": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
#            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",

#negative lookaheadf
#if len re.findall
#([[re.match(s, col) for s in sets_SILAC] for col in test_liast])


#i_acquisition = pn.widgets.Select(options=["LFQ","SILAC"])
#i_class.acquisition= i_acquisition.value
#i_sets = pn.widgets.Select(options=acquisition_set_dic[i_acquisition.value])
#
#@pn.depends(i_acquisition.param.value, watch=True)
#def update_sets(acquisition):
#    acquistion = acquisition_set_dic[acquisition]
#    i_sets.options = acquistion
#

In [ ]:
class SpatialDataSet:

    def __init__(self, **kwargs):
        """Import of the raw file of interest. Dataframe will be generated, which contains only the data of desired
        column names, specified by the dictionary entry regex["col_shortened"]

        Dictionaries are created, that ate used for filtering and plotting, respectively.

        Args:
            filename: raw file obtained by the LFQ/SILAC approach (Protein Groups Files), processed by MaxQuant

        Returns:
            df_original: shortened data frame, contains only the information, which was determined due to
            the values of the key "col_shortened"
        """
        # df_original contains all information of the raw file; tab separated file is imported,
        # without considering comments, marked with #

        #self.filename = "LFQ_proteinGroups.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        #self.filename = "proteinGroupsCOVID.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        #self.filename = "proteinGroups_LFQ_Deep_3_Maps.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        self.filename = "6_deep_maps.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        
        self.map_of_interest = "MAP1" if "map_of_interest" not in kwargs.keys() else kwargs["map_of_interest"]
        self.cluster_of_interest = "Proteasome" if "cluster_of_interest" not in kwargs.keys() else kwargs[
            "cluster_of_interest"]

        self.summed_MSMS_counts = 2 if "summed_MSMS_counts" not in kwargs.keys() else kwargs["summed_MSMS_counts"]
        self.consecutive_LFQ_I = 4 if "consecutive_LFQ_I" not in kwargs.keys() else kwargs["consecutive_LFQ_I"]

        # self.lfq_filter_param = {"summed MS/MS counts": 2, "consecutive LFQ_I": 4}

        self.regex = {
            "imported_columns": "^[Rr]atio H/L (?!normalized|type|is.*).+|id$|[Mm][Ss].*[cC]ount.+$|[Ll][Ff][Qq].*|.*[nN]ames.*|.*[Pp][rR].*[Ii][Dd]s.*|[Pp]otential.[cC]ontaminant|[Oo]nly.[iI]dentified.[bB]y.[sS]ite|[Rr]everse|[Ss]core|[Qq]-[Vv]alue",
#            "index_col_silac": "[Pp]rotein.[Ii][Dd]s|[Mm]ajority.[Pp]rotein.[Ii][Dd]s|[Pp]rotein.[Nn]ames|[Gg]ene.[Nn]ames|[Ii][Dd]|[Ss]core|[Qq]-[Vv]alue",
#            "index_col_lfq": ".*[Pp][rR].*[Ii][Dd]s.*|.*[nN]ames.*|[Ii][Dd]|[Ss]core|[Qq]-[Vv]alue|MS/MS.count$",

#            "type_count_silac": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
#            "type_var_silac": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
#            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",
#
#            "type_count_lfq": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
#            "type_var_lfq": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
#            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",
#
#            "type_msms_lfq": "([Mm][Ss]/[Mm][Ss].[cC]ount)[ .].*_.*K",
#            "type_intensity_lfq": "([Ll][Ff][Qq].[Ii]ntensity)[ .].*_.*K",

            # "type_lfq": "(.*[nNTt]{1}[yYtT]{1})[ .].*_\d+[Kk]$",

            #"lfq_nan": "[Ll][Ff][Qq].*",

            #"contaminants": "[Pp]otential.[cC]ontaminant",
            #"sites": "[Oo]nly.[iI]dentified.[bB]y.[sS]ite",
            #"reverse": "[Rr]everse"
        }
        
        #self.acquisition = "SILAC" if "acquisition" not in kwargs.keys() else kwargs["acquisition"]
        self.acquisition = "LFQ" if "acquisition" not in kwargs.keys() else kwargs["acquisition"]
        
        self.acquisition_set_dict = {
        'LFQ': ["[Ll][Ff][Qq].[Ii]ntensity", "[Mm][Ss]/[Mm][Ss].[cC]ount", "[Ii]ntensity"],
        'SILAC'  : [ "[Rr]atio.[Hh]/[Ll](?!.[Vv]aria|.[Cc]ount)","[Rr]atio.[Hh]/[Ll].[Vv]ariability.\[%\]", "[Rr]atio.[Hh]/[Ll].[cC]ount"]
        }
        
        self.name_pattern = ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)" if "name_pattern" not in kwargs.keys() else kwargs["name_pattern"]
        #name_pattern_LFQ = ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)"
        #name_pattern_SILAC = ".* (?P<rep>.*)_(?P<frac>.*)"
        #name_pattern_SILAC = "(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)"
        
        self.fraction_dict = {"1K": "01K","3K": "03K", "6K": "06K", "12K": "12K", "24K": "24K", "80K": "80K", "01K": "01K","03K": "03K", "06K": "06K", "012K": "12K", "024K": "24K", "080K": "80K", "Cyt": "Cyt", "Mem": "Mem", "Nuc": "Nuc", "Prot": "Prot", "cyt": "Cyt"}
        
        self.analysed_datasets_dict = {}
        self.analysis_summary_dict = {}
        self.shape_dict = {}  
        self.expname = "Protein_Groups" if "expname" not in kwargs.keys() else kwargs["expname"]
            #msms_counts:
        
        
    def data_reading(self):
        """Data import.

        Args:
            filename: stored as attribute

        Returns:
            df_orginal: raw, unprocessed dataframe, single level column index
        """

        self.df_original = pd.read_csv(self.filename, sep="\t", comment="#",
                                       usecols=lambda x: bool(re.match(self.regex["imported_columns"], x)))

        return self.df_original
    
    def processingdf(self):
        """Analysis of the SILAC/LFQ data will be performed.

        The dataframe will be filtered, normalized and converted into a dataframe, characterized by a flat column index,
        which can be used for plotting

        Args:
            acquisition mode: "SILAC" or "LFQ", which is referring to the acquisition method

        Returns:
            A dataframe, in which "Fraction" and "Map" are stacked, containing "normalized profile" as column,
            additionally "Ratio H/L count", "Ratio H/L variability [%]" is found for SILAC data and "MS/MS count"
            for LFQ data; represented as a flat column index
        """
    
        def indexingdf(df_original, acquisition_set_dict, acquisition, fraction_dict, name_pattern, shape_dict):
            # deep copy of the dataframe
            df_original = self.df_original.copy()
            df_i = df_original.set_index([col for col in df_original.columns if any([re.match(s, col) for s in self.acquisition_set_dict[self.acquisition]]) == False])
    
            # multindex will be generated, by isolating the information about the Map, Fraction and Type from each
            # individual column name
            # names=["Set", "Map", "Fraction"] defines the label of the multiindex
            multiindex = pd.MultiIndex.from_arrays(
                    arrays=[
                        [item for sublist in [[re.findall(s, col)[0] for s in self.acquisition_set_dict[self.acquisition] if re.match(s,col)] for col in df_i.columns] for item in sublist],
            
                        [re.match(self.name_pattern, col).group("rep") for col in df_i.columns] 
                        if not "<cond>" in self.name_pattern 
                        else ["_".join(re.match(self.name_pattern, col).group("cond", "rep")) for col in df_i.columns],
                        
                        [self.fraction_dict[re.match(self.name_pattern, col).group("frac")] for col in df_i.columns],
                    ],
                    names=["Set", "Map", "Fraction"]
                        )
            df_i.columns = multiindex
            df_i.sort_index(1, inplace=True)
            
            self.shape_dict["Shape before categorical filtering"]=df_i.shape
            
            df_index = df_i.xs(
                    np.nan, 0, "Reverse").xs(
                    np.nan, 0, "Potential contaminant").xs(
                    np.nan, 0, "Only identified by site")
            
            self.shape_dict["Shape after categorical filtering"]=df_i.shape
    
            return df_index   
    

        def stringency_silac(df_index):
            """The multiindex dataframe is subjected to stringency filtering. Only Proteins with complete profiles are
            considered (a set of f.e. 5 SILAC ratios in case you have 5 fractions / any proteins with missing values
            were rejected). Proteins were retained with 3 or more quantifications in each subfraction (=count). Furthermore,
            proteins with only 2 quantification events in one or more subfraction were retained, if their ratio
            variability for ratios obtained with 2 quantification events was below 30% (=var).
            SILAC ratios were linearly normalized by division through the fraction median. Subsequently normalization
            to SILAC loading was performed.

            Args:
                df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Type

            Returns:
                df_stringency_mapfracstacked: dataframe, in which "MAP" and "Fraction" are stacked;
                the columns "Ratio H/L count", "Ratio H/L variability [%]", and "Ratio H/L" stored as single level indices
            """

            # Fraction and Map will be stacked
            df_stack = df_index.stack(["Fraction", 'Map'])

            len_fractions = len(df_stack.index.get_level_values("Fraction").unique())

            # filtering for sufficient number of quantifications (count in 'Ratio H/L count'), taken
            # variability (var in Ratio H/L variability [%]) into account
            # zip: allows direct comparison of count and var
            # only if the filtering parameters are fulfilled the data will be introduced into df_countvarfiltered_stacked
            df_countvarfiltered_stacked = df_stack.loc[[count >= 3 or (count >= 2 and var < 30) for var, count in
                                                        zip(df_stack["Ratio H/L variability [%]"],
                                                            df_stack['Ratio H/L count'])]]
            
            self.shape_dict["Shape after Ratio H/L count (>= 3)/var (count>=2, var<30) filtering"]=df_countvarfiltered_stacked.shape

            # "Ratio H/L":normalization to SILAC loading, each individual experiment (FractionXMap) will be divided by its median
            # np.median([...]): only entries, that are not NANs are considered
            df_normsilac_stacked = df_countvarfiltered_stacked["Ratio H/L"].unstack(["Fraction", "Map"]).apply(
                lambda x: x / np.median([el for el in x if not np.isnan(el)]), axis=0).stack(["Map", "Fraction"])

            df_stringency_mapfracstacked = df_countvarfiltered_stacked[["Ratio H/L count",
                                                                    "Ratio H/L variability [%]"]].join(
                pd.DataFrame(df_normsilac_stacked, columns=["Ratio H/L"]))

            # dataframe is grouped (Map, id), that allows the filtering for complete profiles
            df_stringency_mapfracstacked = df_stringency_mapfracstacked.groupby(["Map", "id"]).filter(
                lambda x: len(x) >= len_fractions)
            
            self.shape_dict["Shape after filtering for complete profiles"]=df_stringency_mapfracstacked.shape
            
            # Ratio H/L is converted into Ratio L/H
            df_stringency_mapfracstacked["Ratio H/L"] = df_stringency_mapfracstacked["Ratio H/L"].transform(lambda x: 1 / x)


            return df_stringency_mapfracstacked


        def normalization_01_silac(df_stringency_mapfracstacked):
            """The multiindex dataframe, that was subjected to stringency filtering, is 0-1 normalized ("Ratio H/L").

            Args:
                df_stringency_mapfracstacked: dataframe, in which "MAP" and "Fraction" are stacked;
                the columns "Ratio H/L count", "Ratio H/L variability [%]", and "Ratio H/L" stored as single level indices

            Returns:
                df_01_stacked: dataframe, in which "MAP" and "Fraction" are stacked; data in the column
                "Ratio H/L" is 0-1 normalized and renamed to "normalized profile"; the columns "Ratio H/L count",
                "Ratio H/L variability [%]", and "normalized profile" stored as single level indices;
                plotting is possible now
            """

            df_01norm_unstacked = df_stringency_mapfracstacked["Ratio H/L"].unstack("Fraction")

            # 0:1 normalization of Ratio L/H
            df_01norm_unstacked = df_01norm_unstacked.div(df_01norm_unstacked.sum(axis=1), axis=0)

            df_01_stacked = df_stringency_mapfracstacked[["Ratio H/L count", "Ratio H/L variability [%]"]].join(pd.DataFrame
                (
                df_01norm_unstacked.stack(
                    "Fraction"),
                columns=[
                    "Ratio H/L"]))

            # "Ratio H/L" will be renamed to "normalized profile"
            df_01_stacked.columns = [col if col != "Ratio H/L" else "normalized profile" for col in
                                     df_01_stacked.columns]

            return df_01_stacked


        def logarithmization_silac(df_stringency_mapfracstacked):
            """The multiindex dataframe, that was subjected to stringency filtering, is logarithmized ("Ratio H/L").

            Args:
                df_stringency_mapfracstacked: dataframe, in which "MAP" and "Fraction" are stacked;
                the columns "Ratio H/L count", "Ratio H/L variability [%]", and "Ratio H/L" stored as single level indices

            Returns:
                df_log_stacked: dataframe, in which "MAP" and "Fraction" are stacked; data in the column
                "log profile" originates from logarithmized "Ratio H/L" data; the columns "Ratio H/L count",
                "Ratio H/L variability [%]" and  "log profile" are stored as single level indices; PCA is possible now

            """
            # logarithmizing, basis of 2
            df_lognorm_ratio_stacked = df_stringency_mapfracstacked["Ratio H/L"].transform(np.log2)
            df_log_stacked = df_stringency_mapfracstacked[["Ratio H/L count", "Ratio H/L variability [%]"]].join(
                pd.DataFrame
                (df_lognorm_ratio_stacked, columns=["Ratio H/L"]))

            # "Ratio H/L" will be renamed to "log profile"
            df_log_stacked.columns = [col if col != "Ratio H/L" else "log profile" for col in df_log_stacked.columns]

            return df_log_stacked

        
        def stringency_lfq(df_index):
            """The multiindex dataframe is subjected to stringency filtering. Only Proteins which were identified with
            at least [4] consecutive data points regarding the "LFQ intensity", and if summed MS/MS counts >= n(fractions)*[2]
            (LFQ5: min 10 and LFQ6: min 12, respectively; coverage filtering) were included.

            Args:
                df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Typ

            Returns:
                df_stringency_mapfracstacked: dataframe, in which "Map" and "Fraction" is stacked;
                "LFQ intensity" and "MS/MS count" define a single-level column index
                """

            # retrieve number of fractions that are present in the dataset
            df_fractionnumber_stacked = df_index.copy().stack("Fraction")
            number_fractions = len(df_fractionnumber_stacked.index.get_level_values("Fraction").unique())

            df_index = df_index.stack("Map")

            # sorting the level 0, in order to have LFQ intensity -	MS/MS count instead of continuous alternation
            df_index.sort_index(axis=1, level=0, inplace=True)

            # "MS/MS count"-column: take the sum over the fractions;
            # if the sum is larger than n[fraction]*2, it will be stored in the new dataframe
            df_mscount_mapstacked = df_index.loc[df_index[('MS/MS count')].apply(np.sum, axis=1) >= (
                    number_fractions * self.summed_MSMS_counts)]

            self.shape_dict["Shape after MS/MS value filtering"]=df_mscount_mapstacked.shape
            
            df_stringency_mapfracstacked = df_mscount_mapstacked.copy()

            # series no dataframe is generated; if there are at least i.e. 4 consecutive non-NANs, data will be retained
            df_stringency_mapfracstacked = df_stringency_mapfracstacked.loc[
                df_stringency_mapfracstacked[("LFQ intensity")].apply(lambda x: any(
                    np.invert(np.isnan(x)).rolling(window=self.consecutive_LFQ_I).sum() >=
                    self.consecutive_LFQ_I), axis=1)]
            
            self.shape_dict["Shape after consecutive value filtering"]=df_stringency_mapfracstacked.shape

            df_stringency_mapfracstacked = df_stringency_mapfracstacked.copy().stack("Fraction")

            return df_stringency_mapfracstacked


        def normalization_01_lfq(df_stringency_mapfracstacked):
            """The multiindex dataframe, that was subjected to stringency filtering, is 0-1 normalized ("LFQ intensity").

            Args:
                df_stringency_mapfracstacked: dataframe, in which "Map" and "Fraction" is stacked;
                "LFQ intensity" and "MS/MS count" define a single-level column index


            Returns:
                df_01_stacked: dataframe, in which "MAP" and "Fraction" are stacked; data in the column
                "LFQ intensity" is 0-1 normalized and renamed to "normalized profile";
                the columns ""normalized profile"" and "MS/MS count" are stored as
                single level indices; plotting is possible now

            """

            df_01norm_mapstacked = df_stringency_mapfracstacked["LFQ intensity"].unstack("Fraction")

            # 0:1 normalization of Ratio L/H
            df_01norm_unstacked = df_01norm_mapstacked.div(df_01norm_mapstacked.sum(axis=1), axis=0)

            df_01_stacked = df_stringency_mapfracstacked[["MS/MS count"]].join(pd.DataFrame(df_01norm_unstacked.stack(
                   "Fraction"),columns=["LFQ intensity"]))

            # rename columns: "LFQ intensity" into "normalized profile"
            df_01_stacked.columns = [col if col != "LFQ intensity" else "normalized profile" for col in
                                     df_01_stacked.columns]
            df_01_stacked = df_01_stacked.sort_index()
            return df_01_stacked


        def logarithmization_lfq(df_stringency_mapfracstacked):
            """The multiindex dataframe, that was subjected to stringency filtering, is logarithmized ("LFQ intensity").

            Args:
                df_stringency_mapfracstacked: dataframe, in which "Map" and "Fraction" is stacked;
                "LFQ intensity" and "MS/MS count" define a single-level column index

            Returns:
                df_log_stacked: dataframe, in which "MAP" and "Fraction" are stacked; data in the column "log profile"
                originates from logarithmized  "LFQ intensity"; the columns "log profile" and "MS/MS count" are
                stored as single level indices; PCA is possible now
            """

            df_lognorm_ratio_stacked = df_stringency_mapfracstacked["LFQ intensity"].transform(np.log2)
            #df_log_stacked = df_stringency_mapfracstacked.copy()
            #df_log_stacked["LFQ intensity"] = df_lognorm_mapstacked
            #df_log_stacked = df_log_stacked.fillna(0).stack("Fraction")

            df_log_stacked = df_stringency_mapfracstacked[["MS/MS count"]].join(pd.DataFrame
                (df_lognorm_ratio_stacked, columns=["LFQ intensity"]))

            # "Ratio H/L" will be renamed to "log profile"
            df_log_stacked.columns = [col if col != "LFQ intensity" else "log profile" for col in df_log_stacked.columns]

            return df_log_stacked


        if self.acquisition == "SILAC":
            df_index = indexingdf(self.df_original, self.acquisition_set_dict, self.acquisition, self.fraction_dict, self.name_pattern, self.shape_dict)
            df_stringency_mapfracstacked = stringency_silac(df_index)
            df_01_stacked = normalization_01_silac(df_stringency_mapfracstacked)
            df_log_stacked = logarithmization_silac(df_stringency_mapfracstacked)
            self.df_log_stacked = df_log_stacked
            self.df_01_stacked = df_01_stacked
            fractions = df_01_stacked.index.get_level_values("Fraction").unique()
            self.fractions = fractions
            map_names = self.df_01_stacked.index.get_level_values("Map").unique()
            self.map_names = map_names
            
            self.analysis_summary_dict["changes in Shape after filtering"] = self.shape_dict.copy() 
            self.analysis_parameters = {"acquisition" : self.acquisition, "filename" : self.filename}
            self.analysis_summary_dict["Analysis parameters"] = self.analysis_parameters.copy() 
            self.analysed_datasets_dict[self.expname] = self.analysis_summary_dict.copy() 

            self.analysis_summary_dict.clear()
            self.shape_dict.clear()
            self.analysis_parameters.clear() 
            return self.analysed_datasets_dict


        elif self.acquisition == "LFQ":
            df_index = indexingdf(self.df_original, self.acquisition_set_dict, self.acquisition, self.fraction_dict, self.name_pattern, self.shape_dict)
            df_stringency_mapfracstacked = stringency_lfq(df_index)
            df_01_stacked = normalization_01_lfq(df_stringency_mapfracstacked)
            df_log_stacked = logarithmization_lfq(df_stringency_mapfracstacked)
            self.df_log_stacked = df_log_stacked
            self.df_01_stacked = df_01_stacked
            fractions = df_01_stacked.index.get_level_values("Fraction").unique()
            self.fractions = fractions
            map_names = self.df_01_stacked.index.get_level_values("Map").unique()
            self.map_names = map_names
            
            self.analysis_summary_dict["changes in Shape after filtering"] = self.shape_dict.copy() 
            self.analysis_parameters = {"acquisition" : self.acquisition, "filename" : self.filename}
            self.analysis_summary_dict["Analysis parameters"] = self.analysis_parameters.copy() 
            self.analysed_datasets_dict[self.expname] = self.analysis_summary_dict.copy() 
            
            self.analysis_summary_dict.clear()
            self.shape_dict.clear()
            #self.analysis_parameters.clear() 
            return self.analysed_datasets_dict


        else:
            return "I don't know this"    
        
        #df_index = indexingdf(self.df_original)
        #return df_index    
    
    #list_drop_col = [column for column in df_index.columns if not column.startswith("Ratio ")]
    #list_endcount = [column for column in df_index.columns if column.endswith("count")]
    #list_endpct = [column for column in df_index.columns if column.endswith("[%]")]
    #
    #list_drop_col.extend(list_endcount)
    #list_drop_col.extend(list_endpct)
    #
    #df_index = df_index.drop(list_drop_col, axis=1)
            
            # "LFQ intensity"-values: converting 0 into NAN
            # iteration through the column names, and searches for column names (LFQ intensity, deposited as regular
            # expression in the dictionary entry regex["lfq_nan"]
            #df_index[[col for col in df_index.columns if re.match(regex["lfq_nan"], col)]] = \
            #    df_index[[col for col in df_index.columns if re.match(regex["lfq_nan"], col)]].replace(0, np.nan)
        
            
        

In [ ]:
i_expname = pn.widgets.TextInput(name='Experiment Name', placeholder='Enter a string here...')
i_acquisition = pn.widgets.Select(name='Acquisition', options=["LFQ","SILAC"])
i_file = pn.widgets.Select(name='File',options=["6_deep_maps.txt","LFQ_proteinGroups.txt","proteinGroupsCOVID.txt", 
                                              "proteinGroups_LFQ_Deep_3_Maps.txt"])
i_name_pattern = pn.widgets.Select(name='Name patteren',options=[".* (?P<rep>.*_.*)_(?P<frac>.*)",".* (?P<rep>.*)_(?P<frac>.*)", ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)"])

regex_pattern = {
    ".* (?P<rep>.*)_(?P<frac>.*)" : ["MAP1_03K","MAP3_03K"],
    ".* (?P<rep>.*_.*)_(?P<frac>.*)" : ["EGF_rep1_06K","EGF_rep3_06K"],
    ".* (?P<cond>.*)_(?P<frac>.*)_(?P<rep>.*)" : ["Control_Mem_1", "Control_Cyt_1"]
    }

pattern_examples = pn.widgets.Select(name = "Examples", options=regex_pattern[i_name_pattern.value])

@pn.depends(i_name_pattern.param.value, watch=True)
def update_sets(name_pattern):
    example_for_name_pattern = regex_pattern[name_pattern]
    pattern_examples.options = example_for_name_pattern

#i_name_pattern = pn.widgets.Select(options=[".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)",".* (?P<rep>.*)_(?P<frac>.*)"])

pn.Column(pn.Row(i_acquisition, i_file, i_expname), pn.Row(i_name_pattern, pattern_examples))

In [ ]:
double_check= [i_file.value, i_acquisition.value, i_name_pattern.value, i_expname.value]
double_check

In [ ]:
i_class = SpatialDataSet()

In [ ]:
i_class.filename= i_file.value
i_class.acquisition = i_acquisition.value
i_class.name_pattern = i_name_pattern.value
i_class.expname = i_expname.value
i_class.data_reading()
i_class.processingdf()

In [ ]:
{'SILAC_1': {'changes in Shape after filtering': {'Shape before categorical filtering': (10166,
    90),
   'Shape after categorical filtering': (10166, 90),
   'Shape after Ratio H/L count (>= 3)/var (count>=2, var<30) filtering': (165237,
    3),
   'Shape after filtering for complete profiles': (135765, 3)},
  'Analysis parameters': {'acquisition': 'SILAC',
   'filename': '6_deep_maps.txt'}},
 'LFQ_1': {'changes in Shape after filtering': {'Shape before categorical filtering': (7612,
    60),
   'Shape after categorical filtering': (7612, 60),
   'Shape after MS/MS value filtering': (25651, 10),
   'Shape after consecutive value filtering': (25651, 10)},
  'Analysis parameters': {'acquisition': 'LFQ',
   'filename': 'LFQ_proteinGroups.txt'}},
 'LFQ_COVID': {'changes in Shape after filtering': {'Shape before categorical filtering': (10479,
    48),
   'Shape after categorical filtering': (10479, 48),
   'Shape after MS/MS value filtering': (43078, 8),
   'Shape after consecutive value filtering': (43078, 8)},
  'Analysis parameters': {'acquisition': 'LFQ',
   'filename': 'proteinGroupsCOVID.txt'}},
 'LFQ_3': {'changes in Shape after filtering': {'Shape before categorical filtering': (10387,
    42),
   'Shape after categorical filtering': (10387, 42),
   'Shape after MS/MS value filtering': (19047, 14),
   'Shape after consecutive value filtering': (19047, 14)},
  'Analysis parameters': {'acquisition': 'LFQ',
   'filename': 'proteinGroups_LFQ_Deep_3_Maps.txt'}}}

In [ ]:
dictio_all[i_expname.value]=dictio

analysed_dataset = {
    FILENAME_FOR_COMPARISON: {
        data: processed dataframe reduced to available complexes,
        analysis_summary: {
            pgs_before: n,
            pgs_categorical: n,
            ...
            },
        analysis_parameters: {
            acquisition:
            filename:
            msms_counts:
            ...
            },
        spread_table: output_table
        }
}

In [ ]:

#df_index = i_class.df_original.set_index([col for col in i_class.df_original.columns if any([re.match(s, col) for s in sets_LFQ]) == False])

#df_index
#pn.Row(i_file, read_file)

In [ ]:
i_class.shape_dict

In [ ]:
def indexingdf_silac(df_filt, regex):
            """ A multiindex will be generated, characterized by Map, Fraction and Type as level labels,
            allowing the stacking and unstacking of the dataframe;

            Args:
                df_filt: dataframe, that was filtered for matches to the reverse database, matches only identified by
                site, and potential contaminants.

            Returns:
                df_index: multiindex dataframe, which contains 3 level labels: Map (e.g. MAP1, MAP2, ...)
                Fraction (03K, 06K, 12K, 24K, 80K), Type (Ratio H/L count,Ratio H/L variability [%], Ratio H/L),
                rest of the information is stored in the index (Protein IDs, Majority protein IDs,
                Protein names, Gene names, id)
            """
 

In [ ]:
df_index

In [ ]:
def stringency_silac(df_index):
    """The multiindex dataframe is subjected to stringency filtering. Only Proteins with complete profiles are
    considered (a set of f.e. 5 SILAC ratios in case you have 5 fractions / any proteins with missing values
    were rejected). Proteins were retained with 3 or more quantifications in each subfraction (=count). Furthermore,
    proteins with only 2 quantification events in one or more subfraction were retained, if their ratio
    variability for ratios obtained with 2 quantification events was below 30% (=var).
    SILAC ratios were linearly normalized by division through the fraction median. Subsequently normalization
    to SILAC loading was performed.

    Args:
        df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Type

    Returns:
        df_stringency_mapfracstacked: dataframe, in which "MAP" and "Fraction" are stacked;
        the columns "Ratio H/L count", "Ratio H/L variability [%]", and "Ratio H/L" stored as single level indices
    """

    # Fraction and Map will be stacked
    df_stack = df_index.stack(["Fraction", 'Map'])

    len_fractions = len(df_stack.index.get_level_values("Fraction").unique())

    # filtering for sufficient number of quantifications (count in 'Ratio H/L count'), taken
    # variability (var in Ratio H/L variability [%]) into account
    # zip: allows direct comparison of count and var
    # only if the filtering parameters are fulfilled the data will be introduced into df_countvarfiltered_stacked
    df_countvarfiltered_stacked = df_stack.loc[[count >= 3 or (count >= 2 and var < 30) for var, count in
                                                zip(df_stack["Ratio H/L variability [%]"],
                                                    df_stack['Ratio H/L count'])]]

    # "Ratio H/L":normalization to SILAC loading, each individual experiment (FractionXMap) will be divided by its median
    # np.median([...]): only entries, that are not NANs are considered
    df_normsilac_stacked = df_countvarfiltered_stacked["Ratio H/L"].unstack(["Fraction", "Map"]).apply(
        lambda x: x / np.median([el for el in x if not np.isnan(el)]), axis=0).stack(["Map", "Fraction"])

    df_stringency_mapfracstacked = df_countvarfiltered_stacked[["Ratio H/L count",
                                                            "Ratio H/L variability [%]"]].join(
        pd.DataFrame(df_normsilac_stacked, columns=["Ratio H/L"]))

    # dataframe is grouped (Map, id), that allows the filtering for complete profiles
    df_stringency_mapfracstacked = df_stringency_mapfracstacked.groupby(["Map", "id"]).filter(
        lambda x: len(x) >= len_fractions)

    # Ratio H/L is converted into Ratio L/H
    df_stringency_mapfracstacked["Ratio H/L"] = df_stringency_mapfracstacked["Ratio H/L"].transform(lambda x: 1 / x)


    return df_stringency_mapfracstacked

In [ ]:
dfi_1

In [ ]:
df_stringency_mapfracstacked = stringency_silac(dfi_1)

In [ ]:
df_stringency_mapfracstacked

In [ ]:
def stringency_lfq(df_index, selff):
    """The multiindex dataframe is subjected to stringency filtering. Only Proteins which were identified with
    at least [4] consecutive data points regarding the "LFQ intensity", and if summed MS/MS counts >= n(fractions)*[2]
    (LFQ5: min 10 and LFQ6: min 12, respectively; coverage filtering) were included.

    Args:
        df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Typ

    Returns:
        df_stringency_mapfracstacked: dataframe, in which "Map" and "Fraction" is stacked;
        "LFQ intensity" and "MS/MS count" define a single-level column index
        """

    # retrieve number of fractions that are present in the dataset
    df_fractionnumber_stacked = df_index.copy().stack("Fraction")
    number_fractions = len(df_fractionnumber_stacked.index.get_level_values("Fraction").unique())

    df_index = df_index.stack("Map")

    # level 0 = Fraction, level 1 = Type is converted into level 0 = Type, level 1 = Fraction
    #df_index.columns = df_index.columns.swaplevel(0, 1)

    # sorting the level 0, in order to have LFQ intensity -	MS/MS count instead of continuous alternation
    df_index.sort_index(axis=1, level=0, inplace=True)

    # "MS/MS count"-column: take the sum over the fractions;
    # if the sum is larger than n[fraction]*2, it will be stored in the new dataframe
    df_mscount_mapstacked = df_index.loc[df_index[('MS/MS count')].apply(np.sum, axis=1) >= (
            number_fractions * selff.summed_MSMS_counts)]

    df_stringency_mapfracstacked = df_mscount_mapstacked.copy()

    # series no dataframe is generated; if there are at least i.e. 4 consecutive non-NANs, data will be retained
    df_stringency_mapfracstacked = df_stringency_mapfracstacked.loc[
        df_stringency_mapfracstacked[("LFQ intensity")].apply(lambda x: any(
            np.invert(np.isnan(x)).rolling(window=selff.consecutive_LFQ_I).sum() >=
            selff.consecutive_LFQ_I), axis=1)]

    df_stringency_mapfracstacked = df_stringency_mapfracstacked.copy().stack("Fraction")

    return df_stringency_mapfracstacked

In [ ]:
df_stringency_mapfracstacked = stringency_lfq(dfi_1, i_class)
df_stringency_mapfracstacked